<a href="https://colab.research.google.com/github/cayamada/NMR_plots/blob/main/Plots_intensity_decay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit

# Suponha que seus arquivos se chamem 'dataset1.csv' e 'dataset2.csv'
file_id = 'comparison_Gd_2.csv'
file_id2 = 'comparison__Ab_Gd_2.csv'

# Read the CSV file
df1 = pd.read_csv(file_id)
df2 = pd.read_csv(file_id2)

# Excluir a 10ª coluna (se necessário) - adapte conforme necessário
df1 = df1.drop(df1.columns[-1], axis=1)
df2 = df2.drop(df2.columns[-1], axis=1)

# Extrair o número da primeira coluna e converter para inteiro
df1['Numero'] = df1['Dim 1'].str.extract('(\d+)').astype(int)
df2['Numero'] = df2['Dim 1'].str.extract('(\d+)').astype(int)

# Ordenar os DataFrames com base no número extraído
df1 = df1.sort_values(by='Numero')
df2 = df2.sort_values(by='Numero')

# Remover a coluna auxiliar 'Numero' se não for mais necessária
#df1 = df1.drop(columns='Numero')
#df2 = df2.drop(columns='Numero')

# Combinar DataFrames apenas nos resíduos comuns
common_residues = pd.merge(df1, df2, on='Dim 1', how='inner')

# Defina as colunas que representam as concentrações de Gd (0.000, 1.000, 1.500, 2.000, 2.500)
concentration_columns = ['1.000', '1.500', '2.000', '2.500']

# Função exponencial para ajuste
def exponential_func(x, a, b):
    return a * np.exp(b * x)

# Função para calcular a diferença nas inclinações e interceptações
def compare_trendlines(params1, params2):
    slope_diff = params2[1] - params1[1]
    intercept_diff = params2[0] - params1[0]
    return slope_diff, intercept_diff

# Iterar pelos resíduos em "Dim 1" e plotar os valores de intensidade em função das concentrações
for index, row in common_residues.iterrows():
    residue = row['Dim 1']  # O resíduo em "Dim 1"

    # Obter os rótulos corretos para as colunas de intensidade
    intensity_columns1 = [col + '_x' for col in concentration_columns]
    intensity_columns2 = [col + '_y' for col in concentration_columns]

    intensity_values1 = row[intensity_columns1].astype(float)  # Valores de intensidade do primeiro DataFrame
    intensity_values2 = row[intensity_columns2].astype(float)  # Valores de intensidade do segundo DataFrame

    # Ajustar a linha de tendência exponencial usando curve_fit
    params1, _ = curve_fit(exponential_func, np.arange(len(concentration_columns)), intensity_values1)
    params2, _ = curve_fit(exponential_func, np.arange(len(concentration_columns)), intensity_values2)

    # Calcular os valores da linha de tendência exponencial
    trend_values1 = exponential_func(np.arange(len(concentration_columns)), *params1)
    trend_values2 = exponential_func(np.arange(len(concentration_columns)), *params2)

    # Comparar as linhas de tendência
    slope_diff, intercept_diff = compare_trendlines(params1, params2)

    # Configurar o gráfico de linhas com as linhas de tendência exponenciais
    plt.figure(figsize=(10, 5))
    plt.plot(concentration_columns, intensity_values1, marker='o', color='blue', linestyle='-', label='Ptn')
    plt.plot(concentration_columns, intensity_values2, marker='s', color='red', linestyle='-', label='Ptn + Ab')
    plt.plot(concentration_columns, trend_values1, linestyle='--', color='blue', alpha=0.5)
    plt.plot(concentration_columns, trend_values2, linestyle='--', color='red', alpha=0.5)


    # Set the background color to light gray
    sns.set_style("whitegrid", {"axes.facecolor": ".90"})

    # add a gray grid to the plot
    plt.grid(color='white', linestyle='-', linewidth=2)

    plt.xlabel('Concentração de Gd (µM)')
    plt.ylabel('Intensidade do Sinal')
    plt.title(residue)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    # Set the DPI to 300
    plt.rcParams['figure.dpi'] = 300

    # save the plot
    plt.tight_layout()
    plt.savefig('/content/drive/My Drive/CCPNMR/Gd/Plots/' + residue + '.png', dpi=300, bbox_inches='tight')

    # Exibir o gráfico no notebook
    plt.show()

# Lista para armazenar todos os resíduos com suas diferenças nas inclinações
all_residues_with_slope_diff = []

# Iterar pelos resíduos em "Dim 1" e calcular a diferença na inclinação
for index, row in common_residues.iterrows():
    residue = row['Dim 1']  # O resíduo em "Dim 1"

    # Obter os rótulos corretos para as colunas de intensidade
    intensity_columns1 = [col + '_x' for col in concentration_columns]
    intensity_columns2 = [col + '_y' for col in concentration_columns]

    intensity_values1 = row[intensity_columns1].astype(float)  # Valores de intensidade do primeiro DataFrame
    intensity_values2 = row[intensity_columns2].astype(float)  # Valores de intensidade do segundo DataFrame

    # Ajustar a linha de tendência exponencial usando curve_fit
    params1, _ = curve_fit(exponential_func, np.arange(len(concentration_columns)), intensity_values1)
    params2, _ = curve_fit(exponential_func, np.arange(len(concentration_columns)), intensity_values2)

    # Calcular a diferença nas inclinações
    slope_diff, intercept_diff = compare_trendlines(params1, params2)

    # Armazenar os resultados na lista
    all_residues_with_slope_diff.append({
        'Residue': residue,
        'Slope_Diff': slope_diff,
        'Intercept_Diff': intercept_diff
    })

# Criar um DataFrame com todos os resíduos e diferenças nas inclinações
df_all_slope_diff = pd.DataFrame(all_residues_with_slope_diff)

# Caminho para a pasta no Google Drive
google_drive_path = '/content/drive/MyDrive/CCPNMR/Gd/'

# Nome do arquivo CSV
csv_filename = 'all_residues_with_slope_diff.csv'

# Caminho completo para o arquivo CSV
csv_path = google_drive_path + csv_filename

# Salvar o DataFrame em um arquivo CSV
df_all_slope_diff.to_csv(csv_path, index=False)

# Exibir mensagem informando onde o arquivo foi salvo
print(f'O arquivo foi salvo em: {csv_path}')